# 신경망 학습
* 학습: 훈련 데이터로부터 가중지 매개변수의 최적값을 자동으로 획득하는 것
* 손실함수: 신경망이 학습할 수 있도록 해주는 지표   

## 목표
* 손실 함수의 결과값을 가장 작게 만드는 가중치 매개변수를 찾는것

# MSE(Mean Squred Error)
> $E = \frac{1}{2}\Sigma_k(y_k-t_k)^2$

평균 제곱 오차를 기준으로 이에 대한 값이 작을 수록 정확한 모델    
$y_k$가 특정 레이블에 대한 확률값을 저장한 배열 그리고, $t_k$가 원-핫 인코딩된 정답값을 가지고 있는 배열이라 할때 MSE는 0~1사이의 값을 가진다.

In [37]:
import numpy as np

def mean_squared_error(y, t):
  return 0.5 * np.sum((y-t)**2)

# CEE(Cross Entropy Error)
> $E = -\Sigma t_k \log{y_k}$

마찬가지로 $y_k$는 신경망의 출력, $t_k$는 정답 레이블   
교차 엔트로피 오차를 기준으로 이에 대한 값이 작을 수록 정확한 모델

In [38]:
def cross_entropy_error(y, t):
  delta = 1e-7
  return -np.sum(t * np.log(y + delta)) # np.log(0)일 경우 -inf로 발산, 아주 작은 값을 더하여 예외처리

# 미니배치 학습(데이터 1개가 아닌, 훈련 데이터의 묶음을 고려한 손실함수 수식)
> $E = -\frac{1}{N}\Sigma_n \Sigma_k t_{nk} \log{y_{nk}}$

* 데이터가 총 N개라면 $t_{nk}$는 $n$번째 데이터의 $k$번째 값을 의미함
* $y_{nk}$는 신경망의 출력, $t_{nk}$는 정답 레이블
* 마지막에 $N$으로 나눔으로써 정규화

## Mini-batch(미니배치)
전체 훈련 데이터의 대표로서 무작위로 선택한 작은 덩어리(미니배치)
* MNIST 데이터셋의 경우 훈련 데이터가 60000개
* 모든 데이터를 대상으로 손실 함수의 합을 구하기에는 오버헤드가 크다.
* 따라서, 데이터 일부를 추려 전체의 '근사치'로 이용
* 예를들어, 60000장의 훈련 데이터 중에서 반복적으로 100장을 무작위로 뽑아 그 100장만을 사용하여 학습(미니배치 학습) 

In [39]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab_Notebooks')

from sample.dataset.mnist import load_mnist

def get_data():
  (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True) # 정답 레이블 정규화, one-hot-encoding 적용

  print('x_train''s shape:', x_train.shape)
  print('x_test''s shape:', x_test.shape)
  print('t_train''s shape:', t_train.shape)
  print('t_test''s shape:', t_test.shape)

  return x_train, t_train, x_test, t_test

x_train, t_train, x_test, t_test = get_data()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
x_trains shape: (60000, 784)
x_tests shape: (10000, 784)
t_trains shape: (60000, 10)
t_tests shape: (10000, 10)


In [40]:
import numpy as np
# minibatch 학습을 위한 무작위로 훈련 셋 10장 뽑기

'''
np.random.choice(a, size=None, replace=True, p=None)
  Generates a random sample from a given 1-D array
'''
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
print('batch_mask: ', batch_mask)

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

batch_mask:  [37181 13224 24761 38172  8683 27931 49469 30818 21427 12108]


# (배치용) 교차 엔트로피 오차 구현하기
데이터가 하나로 통째로 묶어있는 경우와, 데이터가 배치로 묶여 입력될 경우 모두를 처리할 수 있는 함수

* y가 1차원이라면, reshape로 데이터의 형상을 바꿈
* 배치의 크기로 나눠 정규화 하고 이미지 1장당 평균의 교차 엔트로피 오차를 계산

In [41]:
def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]
  return -np.sum(t * np.log(y + 1e-7)) / batch_size

# 왜 손실함수를 정의하는가?
* 신경망 학습에서는 최적의 매개변수(weight, bias)를 탐색할 때 손실함수의 값을 가능한 한 작게하는 매개변수 값을 찾는다.
* 즉, 손실함수의 최솟값 $\rightarrow$ 미분해서 0이되는 지점
* 미분 값을 단서로 매개변수의 값을 서서히 갱신하는 과정을 반복
* 미분값이 음수면 그 자중치 매개변수를 양의 방향으로 변화시켜 손실 함수의 값을 줄일 수 있음(2차원 비선형 곡선을 생각하면 됨)
* 미분 값이 0이면 가중치 매개변수를 어느쪽으로 움직여도 손실 함수의 값은 달라지지 않는다 $\rightarrow$따라서 그 가중치 매개변수의 갱신을 멈춤   

`
결론적으로, 신경망을 학습할 때 정확도를 지표로 삼아서는 안된다. \
정확도를 지표로 하면 매개변수의 미분이 대부분의 장소에서 0이 되기 때문이다. \
따라서 손실함수를 지표로 삼아야 한다.
`
* 이는 unit step function을 활성화 함수로 사용하지 않는 이유와 마찬가지이다.
* 대부분의 장소에서 미분값이 0이기 때문에, 계단 함수를 이용하면 손실함수를 지표로 삼는게 아무 의미가 없게 된다.(discrete system의 대표적인 함수이기 때문)
* 따라서 sigmoid function $f(x)=\frac{1}{1+\exp^{-x}}$을 이용하여 연속적인 그래프를 사용하여 신경망을 올바르게 학습시킴

# 기울기(Gradient)
> **Example**   
> $f(x_0, x_1)=x_0^2+x_1^2$   
> $(\frac{\partial f}{\partial x_0},\frac{\partial f}{\partial x_1})$ 처럼 모든 변수의 편미분을 **벡터**로 정리한 것을 Gradient(기울기)라고 칭함.   
> 기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향


# 경사 하강법(Gradient Descent)
* $\eta$는 갱신하는 양을 나타냄, 신경망 학습에서 학습률(Learning rate)라고 칭함
* 한 번의 학습으로 얼마만큼 학습해야 할지, 즉 매개변수 값을 얼마나 갱신하느냐를 정하는 것이 학습률

> $x_0=x_0-\eta\frac{\partial f}{\partial x_0}$   
> $x_1=x_1-\eta\frac{\partial f}{\partial x_1}$ 

* 특히나 학습률 값은 `0.01`이나 `0.001` 등 미리 특정 값으로 정해두어야 함.
* 일반적으로 이 값이 너무 크거나 작으면 최솟값으로 찾아갈 수 없음
* 신경망 학습에서 보통 이 학습률 값을 변경하면서 올바르게 학습하고 있는지를 확인
* 학습률이 너무 크면 큰 값으로 발산, 학습률이 너무 작으면 거의 갱신되지 않은 채 반복 종료

In [42]:
def numerical_gradient(f, x):
  h = 1e-4 # 0.0001
  grad = np.zeros_like(x)

  for idx in range(x.size):
    tmp_val = x[idx]

    # f(x+h) 계산
    x[idx] = tmp_val+h
    fxh1 = f(x)

    # f(x-h) 계산
    x[idx] = tmp_val - h
    fxh2 = f(x)

    grad[idx] = (fxh1 - fxh2) / (2 * h)
    x[idx] = tmp_val

  return grad

'''
gradient_descent
<param:
  f: 최적화 하려는 함수
  init_x: 초깃값
  lr: learning rate
  step_num: 경사법에 따른 반복 횟수
>
<return:
  x: 최소지점의 x값
>
'''
def gradient_descent(f, init_x, lr=0.01, step_num=100):
  x = init_x

  for i in range(step_num):
    grad = numerical_gradient(f, x)
    x -= lr * grad
  
  return x

# 학습 알고리즘
전제: 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다.   
> * **1단계(Mini Batch)**: 훈련 데이터 중 일부를 무작위로 가져옴. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실함수 값을 줄이는 것이 목표
> * **2단계(Calculate Gradient)**: 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시  
> * **3단계(Update Prameter)**: 가중치 매개변수를 기울기 방향으로 아주 조금 갱신   
> * **4단계(Repeatition)**: 1~3단계를 반복   

`데이터를 미니배치로 무작위로 선정하기 때문에 확률적 경사 하강법(stocahstic gradient descent; SGD)라고 칭한다.`

# 미니배치 학습 구현

In [45]:
# 2층 신경망 클래스 구현하기
import matplotlib.pyplot as plt
from sample.common.functions import *
from sample.common.gradient import numerical_gradient

class TwoLayerNet:
  '''
  input_size: 입력층의 뉴런 수 
  hidden_size: 은닉층의 뉴런 수
  output_size: 출력층의 뉴런 수
  '''
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
  
  '''
  predict:
  가중치를 토대로 데이터셋과 연산, 
  활성화 함수 적용, 
  예측값 출력
  '''
  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y

  # x: 입력 데이터, t: 정답 레이블
  def loss(self, x, t):
    y = self.predict(x)
    return cross_entropy_error(y, t)

  # 정확도를 출력하는 함수
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0]) # 입력된 데이터에 대하여 일치하는 확률을 구함

    return accuracy

  # 가중치의 기울기를 구하는 함수(편미분)
  # x: 입력 데이터, t: 정답 레이블
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

In [ ]:
x_train, t_train, x_test, t_test = get_data()
train_loss_list = []

# Hyper Parmater
iters_num = 1000 # 가중치를 SGD를 적용하기 위한 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)
  # grad = network.gradient(x_batch, t_batch) # 성능 개선판

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  # 학습 결과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

iteration = range(iters_num)
plt.plot(iteration, train_loss_list)
plt.xlabel('iteration')
plt.ylabel('loss')

x_trains shape: (60000, 784)
x_tests shape: (10000, 784)
t_trains shape: (60000, 10)
t_tests shape: (10000, 10)


# 시험 데이터로 평가하기
* 오버피팅(Overfitting): 훈련 데이터에 포함된 데이터만 제대로 구분하고, 그렇지 않는 데이터는 식별할수 없는것. 즉, 범용적인 능력을 가지지 못하는것

## 1 Epoch 마다 훈련 데이터와 시험 데이터에 대한 정확도 기록
* Epoch: 1에폭은 학습에서 훈련 데이터를 모두 소진했을 때의 횟수에 해당한다. \
훈련 데이터 10000개를 100개의 미니배치로 학습할 경우, 확률적 경사 하강법을 100회 반복하면\
모든 훈련 데이터를 '소진'하기 된다. 이 경우 100회가 1에폭이 된다.

In [ ]:
# 2층 신경망 클래스 구현하기
import matplotlib.pyplot as plt
from sample.common.functions import *
from sample.common.gradient import numerical_gradient

class TwoLayerNet:
  '''
  input_size: 입력층의 뉴런 수 
  hidden_size: 은닉층의 뉴런 수
  output_size: 출력층의 뉴런 수
  '''
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
  
  '''
  predict:
  가중치를 토대로 데이터셋과 연산, 
  활성화 함수 적용, 
  예측값 출력
  '''
  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y

  # x: 입력 데이터, t: 정답 레이블
  def loss(self, x, t):
    y = self.predict(x)
    return cross_entropy_error(y, t)

  # 정확도를 출력하는 함수
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0]) # 입력된 데이터에 대하여 일치하는 확률을 구함

    return accuracy

  # 가중치의 기울기를 구하는 함수(편미분)
  # x: 입력 데이터, t: 정답 레이블
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads


x_train, t_train, x_test, t_test = get_data()
train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

# Hyper Parmater
iters_num = 10000 # 가중치를 SGD를 적용하기 위한 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)
  # grad = network.gradient(x_batch, t_batch) # 성능 개선판

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  # 학습 결과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc.append(test_acc)
    print('train acc, test_acc |' + str(train_acc) + ", " + str(test_acc))

#### plot loss, accuracy ####
iteration = range(iters_num)

plt.subplot(3, 1, 1)
plt.plot(iteration, train_loss_list)
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(3, 1, 2)
plt.plot(iteration, train_acc_list, label='train accuracy')
plt.plot(iteration, test_acc_list, label='test accuracy')
plt.xlabel('iteration')
plt.ylabel('accuracy')
plt.legend(loc='upper right')
plt.grid(True)

plt.subplot(3, 1, 3)


plt.show()